In [1]:
import requests

/Users/chadschmerling/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
url1 = "https://app.deweydata.io/external-api/v3/products/a6be0385-8820-4943-a509-6eac4154b4f6/files/metadata"

url2 = "https://app.deweydata.io/external-api/v3/products/a6be0385-8820-4943-a509-6eac4154b4f6/files"

In [ ]:
params = {"api_key": api_key}
response = requests.get(url, params=params)


In [1]:
import random
import requests
import time

API_KEY = "wPyQR2ZW.neGVFQUDQVrzp0dqwOWAl67JDrxXalNwFUwKsFiyFpEPtvhxtCcbaovi"

def make_get_request(url, params = None, attempt_count = 1, max_attempts = 5):
    """call API using GET request, including retry logic"""
    headers = {'X-API-KEY': API_KEY,
               'Content-Type': 'application/json'}

    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 429 and attempt_count <= max_attempts:
        sleep_time = (2 ** attempt_count) + random.uniform(0, 2)
        print(f'Ratelimit reached, retrying after {sleep_time:.1f} seconds')
        time.sleep(sleep_time)
        return make_get_request(url=url, params=params, attempt_count=attempt_count+1)

    if response.status_code == 400:
        raise Exception(f'Request failed with {response.status_code}')

    if not (response.status_code >= 200 and response.status_code < 300):
        raise Exception(f'Request failed with {response.status_code} and {response.json()}')

    return response

In [5]:
make_get_request(url2)

<Response [200]>

In [11]:
# set API URL
import os 
PRODUCT_API_PATH = url2

# get list of tables
results = make_get_request(url=PRODUCT_API_PATH+'/multi-table-product-metadata')
table_list = [table_dict['table_name'] for table_dict in results.json()['items']]

# loop through all tables
for table_name in table_list:
    # make folder for table if it doesn't exist
    if not os.path.exists(table_name):
        os.makedirs(table_name)
        
    # loop through all API result pages, keeping track of number of downloaded files
    page = 1
    download_count = 0
    while True:
        # get results from API endpoint, using API key for authentication, for a specific page
        results = make_get_request(
            url=PRODUCT_API_PATH,
            params={'page': page, 'table_name': table_name}
        )
        response_json = results.json()

        # for each result page, loop through download links and save to your computer
        for link_data in response_json['download_links']:
            file_name = link_data['file_name']
            print(f"Downloading file {file_name} to folder {table_name}...")
            data = make_get_request(link_data['link'])
            with open(os.path.join(table_name, file_name), 'wb') as file:
                file.write(data.content)
            download_count += 1

        # only continue if there are more result pages to process
        total_pages = response_json['total_pages']
        if page >= total_pages:
            break
        page += 1

    print(f"Successfully downloaded {download_count} files to folder {table_name}.")

Exception: Request failed with 422 and {'error_code': 'VALIDATION_ERROR', 'message': "No tables or not a multi-table product 'Hourly Weather Data'", 'errors': [{'field': '', 'message': "No tables or not a multi-table product 'Hourly Weather Data'"}]}

In [14]:

# set API URL
PRODUCT_API_PATH = url2

# loop through all API result pages, keeping track of number of downloaded files
page = 1
download_count = 0
while True:
    # get results from API endpoint, using API key for authentication, for a specific page
    results = make_get_request(
        url=PRODUCT_API_PATH,
        params={'page': page}
    )
    response_json = results.json()

    # for each result page, loop through download links and save to your computer
    for link_data in response_json['download_links']:
        print(f"Downloading file {link_data['file_name']}...")
        data = make_get_request(link_data['link'])
        with open(link_data['file_name'], 'wb') as file:
            file.write(data.content)
        download_count += 1

    # only continue if there are more result pages to process
    total_pages = response_json['total_pages']
    if page >= total_pages:
        break
    page += 1

print(f"Successfully downloaded {download_count} files.")

Successfully downloaded 64 files.


In [15]:
import gzip

In [16]:
import pandas as pd

In [20]:
ls = []
for i in range(64):
    print(i)
    df = pd.read_csv(f'Hourly_Weather_Data-{i}.csv.gz', compression='gzip')
    ls.append(df)

df_to_save = pd.concat(ls)
df_to_save.to_csv('Mega_Hourly_Weather.csv')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [18]:
df

,CITY_LOCATION_IDENTIFIER__UP_TO_9_ALPHANUMERIC_CHARACTERS_,OBSERVATION_TIME_IN_UTC__STRING_AS_YYYYMMDDHHMM_,OBSERVATION_TIME_IN_LST__LOCAL_STANDARD_TIME___STRING_AS_YYYYMMDDHHMM_,OBSERVATION_TIME_IN_LOCAL_TIME__STRING_AS_YYYYMMDDHHMM_,DAYLIGHT_STATUS__CHARACTER___D_FOR_DAYTIME_ICONS__N_FOR_NIGHTTIME_ICONS_,SKY_DESCRIPTOR__INTEGER___SEE_LOOKUP_,PRECIPITATION_DESCRIPTOR__INTEGER___SEE_LOOKUP_,TEMPERATURE_DESCRIPTOR__INTEGER___SEE_LOOKUP_,ADDITIONAL_AIR_DESCRIPTOR__OPTIONAL___INTEGER___SEE_LOOKUP_,TEMPERATURE__FLOATING_POINT___CELSIUS_,...,_3_HOUR_PRECIPITATION__OPTIONAL___FLOATING__POINT___CENTIMETERS_,_6_HOUR_PRECIPITATION__OPTIONAL___FLOATING__POINT___CENTIMETERS_,_12_HOUR_PRECIPITATION__OPTIONAL___FLOATING__POINT___CENTIMETERS_,_24_HOUR_PRECIPITATION__OPTIONAL___FLOATING__POINT___CENTIMETERS_,_24_HR_MAXIMUM_TEMPERATURE__OPTIONAL___FLOATING_POINT___CELSIUS_,_6_HOUR_MAXIMUM_TEMPERATURE__OPTIONAL___FLOATING_POINT___CELSIUS_,_24_HR_MINIMUM_TEMPERATURE__OPTIONAL___FLOATING_POINT___CELSIUS_,_6_HOUR_MINIMUM_TEMPERATURE__OPTIONAL___FLOATING_POINT___CELSIUS_,SNOW_COVER__OPTIONAL___FLOATING__POINT___CENTIMETERS_,SOLAR_RADIATION__OPTIONAL___INTEGER___MINUTES_
0,KRDU,202409021100,202409020600,202409020700,D,16.0,*,8.0,*,22.8,...,*,*,*,*,*,*,*,*,*,*
1,KRDU,202409021200,202409020700,202409020800,D,16.0,7,8.0,*,22.8,...,*,0,*,6.83,*,22.8,*,22.8,*,*
2,KRDU,202409021300,202409020800,202409020900,D,16.0,7,8.0,*,23.3,...,*,*,*,*,*,*,*,*,*,*
3,KRDU,202409021400,202409020900,202409021000,D,16.0,7,8.0,*,23.9,...,*,*,*,*,*,*,*,*,*,*
4,KRDU,202409021500,202409021000,202409021100,D,15.0,7,10.0,*,25.6,...,0.13,*,*,*,*,*,*,*,*,*
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1646587,KMIS,201908190900,201908190300,201908190400,N,2.0,*,10.0,*,28.0,...,*,*,*,*,*,*,*,*,*,*
1646588,KMIS,201908191000,201908190400,201908190500,N,2.0,*,10.0,*,26.0,...,*,*,*,*,*,*,*,*,*,*
1646589,KMIS,201908191100,201908190500,201908190600,N,2.0,*,10.0,*,26.0,...,*,*,*,*,*,*,*,*,*,*
1646590,KMIS,201908191200,201908190600,201908190700,N,2.0,*,10.0,*,26.0,...,*,*,*,*,*,*,*,*,*,*
